In [ ]:
#Saliência Normalizada do Scanpath (NSS) é uma Métrica baseada em localização. 
#As métricas baseadas em Localização não dependem da distribuição criada pelo processo Gaussiano (geração do GT). 
#O NSS é uma métrica que envolve um mapa de saliência e um conjunto de fixações.
#A ideia é medir os valores de saliência nos locais de fixação ao longo do scanpath de um observador e posteriormente calcular a média deste valor.
#O maior valor do NSS revela um resultado mais satisfatório.

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

q_all = []
p_all = []
out_all = []
media = 0
out = 0

def normalize(x, method='standard', axis=None):
	x = np.array(x, copy=False)
	if axis is not None:
		y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
		shape = np.ones(len(x.shape))
		shape[axis] = x.shape[axis]
		if method == 'standard':
			res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
		elif method == 'range':
			res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
		elif method == 'sum':
			res = x / np.float_(np.sum(y, axis=1).reshape(shape))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	else:
		if method == 'standard':
			res = (x - np.mean(x)) / np.std(x)
		elif method == 'range':
			res = (x - np.min(x)) / (np.max(x) - np.min(x))
		elif method == 'sum':
			res = x / float(np.sum(x))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	return res


def NSS(saliency_map, fixation_map):
    #Normalized Scanpath Saliency
	s_map = np.array(saliency_map, copy=False)
	f_map = np.array(fixation_map, copy=False) > 0.5

	if not np.any(f_map):
		print('Nenhuma fixação para prever')
		return np.nan
      
	if s_map.shape != f_map.shape:
		s_map = resize(s_map, f_map.shape)

	# Normalize saliency map to have zero mean and unit std
	s_map = normalize(s_map, method='standard')
	# Mean saliency value at fixation locations
	return np.mean(s_map[f_map])


if __name__ == '__main__':
    folder = r'...\saliency_all' #Diretório que contenha os mapas de saliência e os grounds truth

    gt_saliency_folder = folder + '\salmap' #mapas de saliencia do modelo
    gt_fixation_folder = folder + '\\fixmap' #mapas de saliencia - ground truth
    #print(gt_saliency_folder)
     
    for filename_sal in glob.glob(os.path.join(gt_saliency_folder,'*.png')):
        #print(filename_sal)
        p= plt.imread(filename_sal)
        p_all.append(p)
        
    for filename_fix in glob.glob(os.path.join(gt_fixation_folder,'*.jpeg')):
        #print(filename_fix)
        q= plt.imread(filename_fix)
        q_all.append(q)    
                   
    for i in range(0,len(p_all)):
        out = NSS(p_all[i],p_all[i])
        out_all.append(out)
    soma = sum(out_all)
    media = soma/len(p_all)
    print (media) 